In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2

def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        # scaling factor = 1.3
        # minimum neighbor = 5
        
        if faces is ():
            return None
        
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face
        
    cap = cv2.VideoCapture(0)  # Changed to 0 for default camera
    id = 1
    img_id = 0
    
    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            
            cv2.imshow("Cropped face", face)
        
        if cv2.waitKey(1)==13 or int(img_id)==200:  # 13 is the ASCII character of Enter
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")

# Call the function
generate_dataset()

<>:12: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
<>:12: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
/var/folders/q1/fyn7446x64g16rk1mnrvswd00000gn/T/ipykernel_13353/59911529.py:12: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
  if faces is ():
2025-03-27 02:07:07.258 python[13353:484950] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-27 02:07:07.258 python[13353:484950] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [6]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
import cv2
from PIL import Image #pip install pillow
import numpy as np    # pip install numpy

In [12]:
import os
import cv2
import numpy as np
from PIL import Image

def train_classifier(data_dir):
    # Check if directory exists
    if not os.path.exists(data_dir):
        raise Exception(f"Directory {data_dir} does not exist")
        
    # Get list of image paths
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]
    
    if len(path) == 0:
        raise Exception(f"No images found in {data_dir}")
    
    faces = []
    ids = []
    
    for image in path:
        try:
            # Open and convert image to grayscale
            img = Image.open(image).convert('L')
            imageNp = np.array(img, 'uint8')
            
            # Extract ID from filename
            id = int(os.path.split(image)[1].split(".")[1])
            
            faces.append(imageNp)
            ids.append(id)
            
        except Exception as e:
            print(f"Error processing image {image}: {str(e)}")
            continue
    
    if len(faces) == 0:
        raise Exception("No faces could be processed")
    
    ids = np.array(ids)
    
    try:
        # Train and save classifier
        clf = cv2.face.LBPHFaceRecognizer_create()
        clf.train(faces, ids)
        clf.write("classifier.xml")
        print("Training completed successfully")
    except Exception as e:
        print(f"Error during training: {str(e)}")

# Call the function
try:
    train_classifier("data")
except Exception as e:
    print(f"Error: {str(e)}")

Training completed successfully


In [13]:
import cv2
import numpy as np
from PIL import Image
import os

In [16]:
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    
    for (x,y,w,h) in features:
        cv2.rectangle(img, (x,y), (x+w,y+h), color, 2)
        
        face_roi = gray_img[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (200, 200))
        
        id, pred = clf.predict(face_roi)
        confidence = int(100*(1-pred/300))
        
        if confidence > 80:
            if id == 1:
                cv2.putText(img, "Gauri", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            if id == 2:
                cv2.putText(img, "Simran", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            if id == 3:
                cv2.putText(img, "Ridhhi", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            if id == 4:
                cv2.putText(img, "Neel", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            
        else:
            cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
    
    return img

# Loading classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Initialize face recognizer
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

# Initialize video capture with default camera (index 0)
video_capture = cv2.VideoCapture(0)  # Changed from 1 to 0

# Check if camera opened successfully
if not video_capture.isOpened():
    print("Error: Could not open camera")
    exit()

while True:
    ret, img = video_capture.read()
    if not ret:
        print("Failed to grab frame")
        break
        
    img = draw_boundary(img, faceCascade, 1.3, 6, (255,255,255), "Face", clf)
    cv2.imshow("Face Detection", img)
    
    if cv2.waitKey(1) == 13:  # Enter key
        break

video_capture.release()
cv2.destroyAllWindows()